In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('info.csv')

In [3]:
df.head()

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,2,2,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,3,2,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,4,2,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground


In [4]:
df.shape

(63888, 9)

### So this is that dataset we have. We need to create some columns and extract few to get the desired data. Eventually we want our data to have columns:

- batting team
- bowling team
- city
- current_score
- balls left
- wickets_left
- current_run_rate
- last five

### Now we already have few columns as we want it in our dataset. Batting team and bowling team data we already have. we also have city column but it has some null values which we need to figure out how to handle that. For rest all we need do some manipulation.

### Now we will start our feature extraction from the city column. To fill the empty values we will use venue column.



In [5]:
df[df['city'].isnull()]['venue'].value_counts()

Dubai International Cricket Stadium        2969
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: venue, dtype: int64

### Here we are checking the values in ‘venue’ column where city column has null values. If we notice carefully the first word in venue is actually the name of the city where the venue exists for e.g. Dubai in Dubai International Cricket Stadium or Melbourne in Melbourne Cricket Ground.



In [6]:
cities=np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x:x[0]),df['city'])

In [7]:
df['city']=cities

In [8]:
df.isnull().sum()

Unnamed: 0          0
match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

### So we store all the first word of venue column in variable named cities and then use it to fill the the city column. Now there are no null values in our dataset. But still there is one more thing left. Our dataset is a ball-by-ball dataset which means if there are 63000 rows that means that many balls have been bowled and played.

In [9]:
df.drop(columns=['Unnamed: 0','venue'],inplace=True)

In [10]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne
...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo


In [11]:
eligible_cities=df['city'].value_counts()[df['city'].value_counts()>600].index.tolist()

### This shows that there are certain cities where very few deliveries have been played. So we can ignore those cities and only consider the ones which have at least 600 deliveries.

In [12]:
df=df[df['city'].isin(eligible_cities)]

### Now our city column is complete. Coming to current_runs column which is very easy to extract from the runs column. A simple cumsum() function (used to find the cumulative sum of a column) will do the work for us.



In [13]:
df['current_score']=df.groupby('match_id').cumsum()['runs']

In [14]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3
...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo,125
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo,125
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo,127


### Now our next target is to create a ‘balls_left’ column for which firstly we would be creating to new columns: ‘overs’ and ‘balls’ which tells us how many overs have been completed and how many balls of the current over has been bowled respectively. The code is very simple for that.

In [15]:
df['over']=df['ball'].apply(lambda x:str(x).split(".")[0])
df['ball_no']=df['ball'].apply(lambda x:str(x).split(".")[1])

In [16]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5
...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo,125,19,3
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo,125,19,4
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,19,5
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo,127,19,6


### Now by using a simple formula we can create a ‘balls_bowled’ column that is how many balls have been bowled. Formula would be

``` balls_bowled = (overs * 6) + balls```

In [17]:
df['balls_bowled']=(df['over'].astype('int')*6) + df['ball_no'].astype('int')

In [18]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo,125,19,3,117
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo,125,19,4,118
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,19,5,119
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo,127,19,6,120


### And now finally we can create our desired column ‘balls_left’ by subtracting balls_bowled from 120 because there are total 120 balls in an innings. sometimes because of extras (wide, no ball …) the ball count exceeds 120 so in such case we can simply give the value of 0.

In [19]:
df['balls_left']=120-df['balls_bowled']
df['balls_left']=df['balls_left'].apply(lambda x:0 if x<0 else x)

In [20]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115
...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo,125,19,3,117,3
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo,125,19,4,118,2
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,19,5,119,1
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo,127,19,6,120,0


### Now if we look at the ‘player_dismissed’ column it has either value 0 or name of the player got out at that particular ball. First we will replace all the names with 1 and then apply the cumsum() function on it so we can get the total wickets gone and we will subtract it from 10 to get the ‘wickets_left’ column.



In [21]:
df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype('int')
df['player_dismissed'] = df.groupby('match_id').cumsum()['player_dismissed']
df['wickets_left'] = 10 - df['player_dismissed']

In [22]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119,10
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118,10
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117,10
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116,10
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,8,Colombo,125,19,3,117,3,2
63884,964,Sri Lanka,Australia,19.4,0,8,Colombo,125,19,4,118,2,2
63885,964,Sri Lanka,Australia,19.5,0,9,Colombo,125,19,5,119,1,1
63886,964,Sri Lanka,Australia,19.6,2,9,Colombo,127,19,6,120,0,1


In [23]:
df['crr']=(df['current_score']*6)/df['balls_bowled']

In [24]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119,10,0.000000
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118,10,0.000000
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117,10,2.000000
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116,10,4.500000
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115,10,3.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,8,Colombo,125,19,3,117,3,2,6.410256
63884,964,Sri Lanka,Australia,19.4,0,8,Colombo,125,19,4,118,2,2,6.355932
63885,964,Sri Lanka,Australia,19.5,0,9,Colombo,125,19,5,119,1,1,6.302521
63886,964,Sri Lanka,Australia,19.6,2,9,Colombo,127,19,6,120,0,1,6.350000


### Now we need a column that has total runs scored in last five overs. Obviously we will have null values in this column for first 5 overs.



In [25]:
groups=df.groupby('match_id')

match_ids=df['match_id'].unique()
last_five=[]
for id in match_ids:
    last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs'].values.tolist())

In [26]:
df['last_five']=last_five

In [27]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119,10,0.000000,NaN
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118,10,0.000000,NaN
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117,10,2.000000,NaN
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116,10,4.500000,NaN
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115,10,3.600000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,8,Colombo,125,19,3,117,3,2,6.410256,32.0
63884,964,Sri Lanka,Australia,19.4,0,8,Colombo,125,19,4,118,2,2,6.355932,32.0
63885,964,Sri Lanka,Australia,19.5,0,9,Colombo,125,19,5,119,1,1,6.302521,32.0
63886,964,Sri Lanka,Australia,19.6,2,9,Colombo,127,19,6,120,0,1,6.350000,33.0


### Now we have to create a last column which would be our target column. Total runs scored in that innings.


In [28]:
final_df=df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')

In [29]:
final_df=final_df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five','runs_x']]

In [30]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
0,Australia,Sri Lanka,Melbourne,0,119,10,0.000000,NaN,168
1,Australia,Sri Lanka,Melbourne,0,118,10,0.000000,NaN,168
2,Australia,Sri Lanka,Melbourne,1,117,10,2.000000,NaN,168
3,Australia,Sri Lanka,Melbourne,3,116,10,4.500000,NaN,168
4,Australia,Sri Lanka,Melbourne,3,115,10,3.600000,NaN,168
...,...,...,...,...,...,...,...,...,...
50496,Sri Lanka,Australia,Colombo,125,3,2,6.410256,32.0,128
50497,Sri Lanka,Australia,Colombo,125,2,2,6.355932,32.0,128
50498,Sri Lanka,Australia,Colombo,125,1,1,6.302521,32.0,128
50499,Sri Lanka,Australia,Colombo,127,0,1,6.350000,33.0,128


### Now we will drop all the columns which we dont want to have for our model and keep those which we created just now. Also we will shuffle the data to avoid any kind of bias.

In [31]:
final_df.isnull().sum()

batting_team         0
bowling_team         0
city                 0
current_score        0
balls_left           0
wickets_left         0
crr                  0
last_five        12024
runs_x               0
dtype: int64

In [32]:
final_df.dropna(inplace=True)

In [33]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
runs_x           0
dtype: int64

In [34]:
final_df=final_df.sample(final_df.shape[0])

In [35]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
31833,Pakistan,New Zealand,Pallekele,145,19,6,8.613861,35.0,177
44680,South Africa,Australia,Cape Town,154,9,7,8.324324,41.0,178
18021,India,Pakistan,Johannesburg,145,5,5,7.565217,38.0,157
18591,West Indies,South Africa,Johannesburg,79,35,5,5.576471,32.0,131
5824,Sri Lanka,Bangladesh,Colombo,143,36,7,10.214286,50.0,214
...,...,...,...,...,...,...,...,...,...
43767,New Zealand,South Africa,Durban,95,54,9,8.636364,39.0,151
39237,Sri Lanka,South Africa,Chittagong,74,71,8,9.061224,41.0,165
25237,West Indies,India,Barbados,27,88,10,5.062500,25.0,169
10338,India,Bangladesh,Nagpur,58,67,8,6.566038,33.0,174


### With this we end out feature extraction part of the project. So after lot of work we finally have the exact required data we wanted at the start.

### So lets now begin with model building process. For that first we will divide our dataset in training set and testing set using train_test_split module of sklearn library



In [36]:
#Train-Test-Split

X=final_df.drop(columns=['runs_x'])
y=final_df['runs_x']
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [37]:
X_train

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five
22484,India,New Zealand,Christchurch,151,6,3,7.947368,48.0
12653,Sri Lanka,Pakistan,Lahore,146,19,5,8.673267,41.0
29822,Australia,India,Melbourne,69,60,6,6.900000,30.0
11630,India,West Indies,Lauderhill,61,77,10,8.511628,51.0
24399,Australia,Bangladesh,Barbados,59,52,5,5.205882,22.0
...,...,...,...,...,...,...,...,...
41735,Bangladesh,Sri Lanka,Chittagong,50,71,5,6.122449,15.0
4197,Pakistan,New Zealand,Mount Maunganui,95,47,8,7.808219,42.0
195,Australia,Sri Lanka,Adelaide,101,53,9,9.044776,38.0
33025,Pakistan,India,Colombo,57,65,6,6.218182,23.0


### Some preprocessing steps are required here. We will apply one hot encoding on the categorical features (batting_team, bowling_team and city) then we will create a pipleline which would be having our ml model. Also we will apply scaling on our data so that all values come in one range.


### Here four our model I will be using ```xgboost algorithm.```

In [38]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

In [39]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

In [40]:
pipe=Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=300,learning_rate=0.2,max_depth=12,random_state=1))

])

In [41]:
pipe.fit(X_train,Y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', StandardScaler()),
                ('step3',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, early_stopping_rou...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=0.2,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=12, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=300,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=1, ...))])

In [42]:
Y_pred=pipe.predict(X_test)

In [43]:
print(r2_score(Y_test,Y_pred))
print(mean_absolute_error(Y_test,Y_pred))

0.9852130706495486
1.7441243461164764


In [44]:
import pickle

In [45]:
pickle.dump(pipe,open('pipe.pkl','wb'))

In [47]:
eligible_cities

['Colombo',
 'Mirpur',
 'Johannesburg',
 'Dubai',
 'Auckland',
 'Cape Town',
 'London',
 'Pallekele',
 'Barbados',
 'Sydney',
 'Melbourne',
 'Durban',
 'St Lucia',
 'Wellington',
 'Lauderhill',
 'Hamilton',
 'Centurion',
 'Manchester',
 'Abu Dhabi',
 'Mumbai',
 'Nottingham',
 'Southampton',
 'Mount Maunganui',
 'Chittagong',
 'Kolkata',
 'Lahore',
 'Delhi',
 'Nagpur',
 'Chandigarh',
 'Adelaide',
 'Bangalore',
 'St Kitts',
 'Cardiff',
 'Christchurch',
 'Trinidad']